# Tutorial

### Instructions

In this tutorial, you will be creating a classifcation model. The data were collected from the Taiwan Economic Journal for the years 1999 to 2009. Company bankruptcy was defined based on the business regulations of the Taiwan Stock Exchange. Apply what you have learnt and come up with the optimal model to predict if a company goes bankrupt or not. 

Bonus: Determine what are the key features that influence your model. 

 PS: If you find a technique/method that you believe was not shared in class but useful; you can and should use it. No points will be deducted for trying! Do not delete your trial scripts, these will be important for you to understand where you improved from your previous attempts.



In [ ]:
data = pd.read_csv('/Users/nikgurunathan/Downloads/data.csv')  # Replace with your data path
X = data.drop('target', axis=1)  # Replace 'target' with the actual target column name
y = data['target']

# Step 3: Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Step 4: Scale the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Step 5: Model selection and training (example with RandomForest)
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)

# Step 6: Hyperparameter tuning (optional)
# param_grid = {'n_estimators': [100, 200], 'max_depth': [10, 20, None]}
# grid_search = GridSearchCV(rf_model, param_grid, cv=5, scoring='roc_auc')
# grid_search.fit(X_train, y_train)

# Step 7: Evaluate the model
y_pred = rf_model.predict(X_test)
print(classification_report(y_test, y_pred))
print("ROC-AUC Score:", roc_auc_score(y_test, rf_model.predict_proba(X_test)[:, 1]))

# Plotting the ROC curve
fpr, tpr, thresholds = roc_curve(y_test, rf_model.predict_proba(X_test)[:, 1])
plt.plot(fpr, tpr, label="ROC curve (area = %0.2f)" % roc_auc_score(y_test, rf_model.predict_proba(X_test)[:, 1]))
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc="lower right")
plt.show()